In [ ]:
#Set-up code
!pip install pandas
!pip install scipy
!pip install sklearn
!pip install openpyxl
!pip install numpy
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: 

In [ ]:
from pandas.core.arrays import categorical
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Conv1D, MaxPooling1D, Embedding, LSTM, Reshape
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.metrics import Precision, Recall, BinaryAccuracy
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn import preprocessing
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

from scipy.stats import zscore
import matplotlib.pyplot as plt

In [ ]:
# Load the datasets
ratings = pd.read_csv('sample_data/ml_100k.csv')
empty_cols = [col for col in ratings.columns if ratings[col].isnull().all()]

if len(empty_cols) > 0:
    # Remove the empty column
    ratings = ratings.drop(columns=empty_cols)

ratings.columns = ['userId', 'itemId', 'rating']
print(ratings)

       userId  itemId  rating
0         186     302       3
1          22     377       1
2         244      51       2
3         166     346       1
4         298     474       4
...       ...     ...     ...
99994     880     476       3
99995     716     204       5
99996     276    1090       1
99997      13     225       2
99998      12     203       3

[99999 rows x 3 columns]


In [ ]:
# Create a binary column indicating whether a rating is suspicious (1) or not (0)
#ratings['suspicious'] = np.where(ratings['rating'] >= 4, 0, 1)
#print(ratings)

mean = np.mean(ratings['rating'])
std = np.std(ratings['rating'])
ratings['z-score'] = (ratings['rating'] - mean)/std

threshold = 2.0
ratings['sus'] = np.where(ratings['z-score'].abs() > threshold, 1, 0)



user_encoder = preprocessing.LabelEncoder()
user_encoder.fit(ratings['userId'])
item_encoder = preprocessing.LabelEncoder()
ratings['userId'] = user_encoder.transform(ratings['userId'])
ratings['itemId'] = item_encoder.fit_transform(ratings['itemId'])


In [ ]:
# Split the dataset into input features and labels
X = ratings[['userId', 'itemId', 'rating']].values
y = ratings['sus'].values

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape the input features for the LSTM model
timesteps = 1
input_dim = 3
X_train = np.reshape(X_train, (X_train.shape[0], timesteps, input_dim))
X_test = np.reshape(X_test, (X_test.shape[0], timesteps, input_dim))

# Convert the labels to categorical format
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Define the CNN model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(1, 3)))
model.add(Conv1D(filters=64, kernel_size=1, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Reshape for LSTM
model.add(Reshape((1, -1))) # -1 to infer the number of features


# output_size = 1
# Add LSTM layers
# Add LSTM layers
model.add(LSTM(64, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(LSTM(32, activation="sigmoid", return_sequences=True))

# Reshape the output of the second LSTM layer
model.add(Reshape((-1, 32)))

# Add BatchNormalization and Dropout layers
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Add a fully connected layer with sigmoid activation
#model.add(Dense(1, activation='sigmoid'))
model.add(Reshape((-1, 32)))
model.add(Dense(1))



In [ ]:

#assuming X and y are defined
y = y[:99999]

X = X.reshape(X.shape[0], 1, 3)


callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, min_delta = 0.01)

#compile model
#avoid exploding/vanishing gradient problem by setting clipoff

#Compiling the model with the Precision, Recall, and F1 score metrics does not work 


#WORKING BUT ONLY RMSE AND MAE
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['RootMeanSquaredError', 'MeanAbsoluteError'])



 # fit model
model.fit(
      X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32,callbacks=[callback]
 )

625/625 [==============================] - 1s 2ms/step
MAE:  0.060850005596876144
RMSE:  0.2466779202222824
F1 Score:  0.9391499757766724
Precision:  0.9391499757766724
Recall:  0.9391499757766724


In [ ]:
#Evaluate the Model
#WORKING FOR RMSE AND MAE
model.summary()
results = model.evaluate(X_test, y_test, verbose=0)
print('RMSE:',results[1],'\n','MAE:',results[2])

In [ ]:
# Make predictions on test data
y_pred = model.predict(X_test)

# Convert predictions to binary (0 or 1)
y_pred_binary = (y_pred > 0.5).astype(int)

In [ ]:
# converting the nested arrays into a list
y_pred_merged = np.concatenate(y_pred_binary)
list_y_pred_merged = y_pred_merged.tolist()
list_merged = []
for arr in list_y_pred_merged:
    for elem in arr:
        list_merged.append(elem)
print(list_merged)
y_pred_binary1= list_merged

In [ ]:
y_test= y_test.tolist()
y_test1 = [x[0] for x in y_test]

print(y_test1)

In [ ]:
print(len(X_test),len( y_pred_binary1),len(y_test1))

In [ ]:
precision = precision_score(y_test1, y_pred_binary1)
recall = recall_score(y_test1, y_pred_binary1)
f1 = f1_score(y_test1, y_pred_binary1, average='weighted', labels=np.unique(y_pred_binary1))

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
#load infected dataset
ratings_infected = pd.read_csv('sample_data/amazon_50k_bandwagon.csv')
empty_cols = [col for col in ratings_infected.columns if ratings_infected[col].isnull().all()]

if len(empty_cols) > 0:
    # Remove the empty column
    ratings_infected = ratings_infected.drop(columns=empty_cols)

ratings_infected.columns = ['userId', 'itemId', 'rating']
print(ratings_infected)



mean_infected = np.mean(ratings_infected['rating'])
std_infected = np.std(ratings_infected['rating'])
ratings_infected['z-score'] = (ratings_infected['rating'] - mean_infected)/std_infected

threshold = 2.0
ratings_infected['sus'] = np.where(ratings_infected['z-score'].abs() > threshold, 1, 0)
print(ratings_infected)

user_encoder_i = preprocessing.LabelEncoder()
user_encoder_i.fit(ratings_infected['userId'])
item_encoder_i = preprocessing.LabelEncoder()
ratings_infected['userId'] = user_encoder_i.transform(ratings_infected['userId'])
ratings_infected['itemId'] = item_encoder_i.fit_transform(ratings_infected['itemId'])
print('------------------encoded user and item IDs---------------------')
print(ratings_infected)

# Split the dataset into input features and labels
Xi = ratings[['userId', 'itemId', 'rating']].values
yi = ratings['sus'].values

# Split the dataset into training and test sets
X_train_infected, X_test_infected, y_train_infected, y_test_infected = train_test_split(Xi, yi, test_size=0.2, random_state=42)

# Normalize the input features
scaler = StandardScaler()
X_train_infected = scaler.fit_transform(X_train_infected)
X_test_infected = scaler.transform(X_test_infected)

# Reshape the input features for the LSTM model
timesteps = 1
input_dim = 3
X_train_infected = np.reshape(X_train_infected, (X_train_infected.shape[0], timesteps, input_dim))
X_test_infected = np.reshape(X_test_infected, (X_test_infected.shape[0], timesteps, input_dim))

# Convert the labels to categorical format
num_classes = 2
y_train_infected = keras.utils.to_categorical(y_train_infected, num_classes)
y_test_infected = keras.utils.to_categorical(y_test_infected, num_classes)


# Evaluate the model on the test set
resultsi = model.evaluate(X_test_infected, y_test_infected, batch_size=100)
print('RMSE:',resultsi[1],'\n','MAE:',resultsi[2])

# Calculate predictions
y_pred_infected = model.predict(X_test_infected)

# Convert predictions to binary format
y_pred_infected = (y_pred_infected > 0.5).astype(int)

# converting the nested arrays into a list
y_pred_i_merged = np.concatenate(y_pred_infected)
list_y_pred_i_merged = y_pred_i_merged.tolist()
list_i_merged = []
for arr in list_y_pred_i_merged:
    for elem in arr:
        list_i_merged.append(elem)
print(list_i_merged)
y_pred_i_binary1= list_i_merged

y_test_infected= y_test_infected.tolist()
y_test_infected1 = [x[0] for x in y_test_infected]

print(y_test_infected1)

precision_i = precision_score(y_test_infected1, y_pred_i_binary1)
recall_i = recall_score(y_test_infected1, y_pred_i_binary1)
f1_i = f1_score(y_test_infected1, y_pred_i_binary1)

print("Precision:", precision_i)
print("Recall:", recall_i)
print("F1 Score:", f1_i)

        userId  itemId  rating
0          344     216       4
1          344     215       3
2          344     213       4
3          344     210       4
4          344     762       3
...        ...     ...     ...
109921    1059     195       4
109922    1059     541       4
109923    1059     807       4
109924    1059     153       4
109925    1059     804       4

[109926 rows x 3 columns]
        userId  itemId  rating   z-score  sus
0          344     216       4  0.404923    0
1          344     215       3 -0.505427    0
2          344     213       4  0.404923    0
3          344     210       4  0.404923    0
4          344     762       3 -0.505427    0
...        ...     ...     ...       ...  ...
109921    1059     195       4  0.404923    0
109922    1059     541       4  0.404923    0
109923    1059     807       4  0.404923    0
109924    1059     153       4  0.404923    0
109925    1059     804       4  0.404923    0

[109926 rows x 5 columns]
------------------enco

In [ ]:
tf.compat.v1.reset_default_graph()